---
# Introdução

#### Esse notebook faz parte do curso "[Introduction to Data Science in Python da Universidade de Michigan](https://www.coursera.org/learn/python-data-analysis/home/welcome)".


Nessa atividade (Assigment 4 - Statistical Analysis in Python), devemos realizar um teste de hipótese se as cidades universitárias (EUA) têm seus os preços médios de residências menos afetados durante recessões.

As questões estão separadas por funções, e os comentários e discussões sobre a resolução de cada questão se encontram em _markdown_ logo acima de cada questão, de maneira didática.

---

---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [26]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

---

## Certo! Temos algumas definições dadas pelo problema: 
- um *quarter* (trimestre) corresponde a um período de 3 meses:
    - Q1 é de janeiro a março
    - Q2 é de abril a junho
    - Q3 é de julho a setembro
    - Q4 é de outubro a dezembro
    
- uma *recession* (recessão) corresponde a começar com 2 trimestres consecutivos de GDP (PIB) e com trimestre final em ascenção de PIB.

- uma *recession bottom* é o trimestre que obteve o menor PIB da recessão.

- uma *university town* (cidade universitária) é uma cidade que possui maior porcentagem de universitários comparada com a população da cidade.

## O conjunto de dados:

1. No Zillow.com, existem dados de moradia para os Estados Unidos. Em particular, o arquivo de dados de todas as residências no nível da cidade, esse arquivo (`City_Zhvi_AllHomes.csv`) tem preços médios de venda de residências em um nível refinado.  

2. Na página da Wikipedia sobre cidades universitárias, há uma lista de cidades universitárias nos Estados Unidos que foram copiadas e coladas no arquivo `university_towns.txt`. 

3. Do Bureau of Economic Analysis, Departamento de Comércio dos EUA, as horas extras do PIB dos Estados Unidos em dólares correntes, em intervalos trimestrais, no arquivo `gdplev.xls`.

## Hipótese: 

As cidades universitárias têm seus preços médios de moradia menos afetados pelas recessões. Faça um teste t para comparar a proporção do preço médio das casas nas cidades universitárias no trimestre anterior ao início da recessão, em comparação com o nível mais baixo da recessão. (price_ratio = quarter_before_recession / recession_bottom)


# E vamos para nossa primeira questão: Retornar uma lista contendo todas as cidades universitárias

`university_towns` trata-se de um ``.txt`` que tem o formato de Estado [editar]  Nome da cidade  Nome da universidade (Número) - Nome da cidade - Nome da universidade (Número). Portanto, a primeira tarefa é obter um _dataframe_ contendo o nome do estado e as Cidades da Universidade nesse estado.

### Algumas dicas para limpar os dados:

1. Para "State", remova os caracteres de "[" até o fim.

2. Para "RegionName", quando aplicável, remova todos os caracteres de "(" até o final.

3. Dependendo de como você lê os dados, pode ser necessário remover o caractere de nova linha '\ n'.

Pode haver alguns RegionNames na lista que não correspondam bem às cidades reais, mas não se preocupe em fazer algo especial para essas entradas para os fins desta tarefa. A pontuação no final do RegionName não encapsulada entre parênteses não precisa ser removida.

Ok! Vamos lá

Irei primeiramente carregar o arquivo (pode ser que haja uma maneira mais *pandorable* de se fazer isso, mas por se tratar de um .txt vamos de ```.open``` ) para ```file_txt``` percorrer cada linha e armazenar o *state* atual.

Para pegar o state atual, podemos verificar se a string contém um 'edit' como substring. Dentro disso, temos as regioes que podemos ignorar tudo depois da '(' também.

Devemos atentar para duas coisas ao splitar as strings.
   
   - 1.  Atentar pro espaço antes do parênteses. Isso refere-se que se o split for em '(' e não em ' (', um espaço em branco ficará na posição 0 do array da string.
   - 2. Atentar para linhas que não contenham '(', nesse caso, podemos usar o splitline, que separa em \n


Podemos adicionar os valores em 2 listas para transformarmos em dataframe posteriormente.

In [4]:
file_txt = open("university_towns.txt")
state = []
region = []
for line in file_txt:
    print(line)

Alabama[edit]

Auburn (Auburn University)[1]

Florence (University of North Alabama)

Jacksonville (Jacksonville State University)[2]

Livingston (University of West Alabama)[2]

Montevallo (University of Montevallo)[2]

Troy (Troy University)[2]

Tuscaloosa (University of Alabama, Stillman College, Shelton State)[3][4]

Tuskegee (Tuskegee University)[5]

Alaska[edit]

Fairbanks (University of Alaska Fairbanks)[2]

Arizona[edit]

Flagstaff (Northern Arizona University)[6]

Tempe (Arizona State University)

Tucson (University of Arizona)

Arkansas[edit]

Arkadelphia (Henderson State University, Ouachita Baptist University)[2]

Conway (Central Baptist College, Hendrix College, University of Central Arkansas)[2]

Fayetteville (University of Arkansas)[7]

Jonesboro (Arkansas State University)[8]

Magnolia (Southern Arkansas University)[2]

Monticello (University of Arkansas at Monticello)[2]

Russellville (Arkansas Tech University)[2]

Searcy (Harding University)[5]

California[edit]

Angw

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    file_txt = open("university_towns.txt")
    state = []
    region = []
    for line in file_txt:
        if 'edit' in line:
            current_state = line.split('[')[0]
            continue

        state.append(current_state) 
        region.append(line.split(' (')[0] if '(' in line else line.splitlines()[0])

    df_unitown = pd.DataFrame(list(zip(state, region)), columns=['State', 'RegionName'])
    
    return df_unitown
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


# Segunda questão: Identificar o ano e trimestre em que ocorreu a recessão.

 - Uma *recession* (recessão) corresponde a começar com 2 trimestres consecutivos de GDP (PIB) e com trimestre final em ascenção de PIB.

 - Precisamos retornar o ano e o trimestre como um valor string.

- Devemos atentar para as dicas para o exercício:
    
   - use o valor encadeado em dólares de 2009 
   - Para esta tarefa, observe apenas os dados do PIB a partir do primeiro trimestre de 2000.

Para começar, precisamos carregar o ```gdplev.xls```.

Podemos utilizar a função `cut` para sabermos o semestre.

In [11]:
trim_df = pd.read_excel('gdplev.xls', header=None)
trim_df.head(10)

,0,1,2,3,4,5,6,7
0,"Current-Dollar and ""Real"" Gross Domestic Product",NaN,NaN,NaN,NaN,NaN,NaN,2016-09-29
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,Annual,NaN,NaN,NaN,Quarterly,NaN,NaN,NaT
3,NaN,NaN,NaN,NaN,(Seasonally adjusted annual rates),NaN,NaN,NaT
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
5,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaN,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaT
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8,1929,104.6,1056.6,NaN,1947q1,243.1,1934.5,NaT
9,1930,92.2,966.7,NaN,1947q2,246.3,1932.3,NaT


OK, temos um problema. Colunas não estão corretamente nomeadas e os dados só começam a contar apartir da linha 9.

Vamos corrigir isso.

Primeiramente vamos skipar as primeiras 8 linhas, e vamos olhar para as colunas (seasonally adjusted annual rates) = E, e  (GDP in bilions of chained 2009 dollars) G.

In [12]:
trim_df = pd.read_excel('gdplev.xls', skiprows=9, parse_cols='E,G', header=None)
trim_df.columns = ['quarter', 'GDP']
trim_df.head(10)

,quarter,GDP
0,1947q2,1932.3
1,1947q3,1930.3
2,1947q4,1960.7
3,1948q1,1989.5
4,1948q2,2021.9
5,1948q3,2033.2
6,1948q4,2035.3
7,1949q1,2007.5
8,1949q2,2000.8
9,1949q3,2022.8


Certo, mas precisamos olhar apenas para os dados a partir do ano 2000.

Vamos fazer isso splitando o ano tirando o 'q', e selecionando apenas os que sao acima de 2000.

In [13]:
trim_df['date'] = trim_df['quarter'].apply(lambda x: x.split('q')[0]).astype(int)
pib_df = trim_df[trim_df['date'] >= 2000]
pib_df.head()

,quarter,GDP,date
211,2000q1,12359.1,2000
212,2000q2,12592.5,2000
213,2000q3,12607.7,2000
214,2000q4,12679.3,2000
215,2001q1,12643.3,2001


Certo, agora vamos eliminar a coluna criada 'date'.

In [14]:
pib_df.drop('date', axis=1, inplace=True)

In [15]:
pib_df.reset_index(inplace=True)

OK, agora precisamos retornar os anos de recessões, lembrando que uma recession (recessão) corresponde a começar com 2 trimestres consecutivos de GDP (PIB) em declínio (q1 > q2 e q2 > q3) seguidos por trimestres finais em ascenção de PIB (q3 < q4), ou seja, dado um trimestre x, queremos que x-2 seja > que x-1, e que x-1 seja > que x

In [16]:
for i in range(0, pib_df.shape[0]-4):
        if ((pib_df.loc[i, 'GDP'] > pib_df.loc[i+1, 'GDP']) and 
            (pib_df.loc[i+1, 'GDP'] > pib_df.loc[i+2, 'GDP']) and 
            (pib_df.loc[i+2, 'GDP'] < pib_df.loc[i+3, 'GDP'])):
            (pib_df.loc[i+3, 'GDP'] < pib_df.loc[i+4, 'GDP'])
            idx = i
            startdate = pib_df.loc[i, 'quarter']

startdate

'2008q4'

In [17]:
print(pib_df.loc[idx-1, 'quarter'], pib_df.loc[idx, 'GDP'])
print(pib_df.loc[idx, 'quarter'], pib_df.loc[idx, 'GDP'])
print(pib_df.loc[idx+1, 'quarter'], pib_df.loc[idx+1, 'GDP'])
print(pib_df.loc[idx+2, 'quarter'], pib_df.loc[idx+2, 'GDP'])
print(pib_df.loc[idx+3, 'quarter'], pib_df.loc[idx+3, 'GDP'])
print(pib_df.loc[idx+4, 'quarter'], pib_df.loc[idx+4, 'GDP'])

            
print(pib_df.loc[idx-1, 'quarter'])

2008q3 14577.0
2008q4 14577.0
2009q1 14375.0
2009q2 14355.6
2009q3 14402.5
2009q4 14541.9
2008q3


In [93]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    trim_df = pd.read_excel('gdplev.xls', skiprows=9, parse_cols='E,G', header=None)
    trim_df.columns = ['quarter', 'GDP']
    trim_df['date'] = trim_df['quarter'].apply(lambda x: x.split('q')[0]).astype(int)
    
    pib_df = trim_df[trim_df['date'] >= 2000]
    pib_df.drop('date', axis=1, inplace=True)
    pib_df.reset_index(inplace=True)
    
    for i in range(0, pib_df.shape[0]-4):
        if ((pib_df.loc[i, 'GDP'] > pib_df.loc[i+1, 'GDP']) and 
            (pib_df.loc[i+1, 'GDP'] > pib_df.loc[i+2, 'GDP']) and 
            (pib_df.loc[i+2, 'GDP'] < pib_df.loc[i+3, 'GDP'])):
            (pib_df.loc[i+3, 'GDP'] < pib_df.loc[i+4, 'GDP'])
            idx = i

    return pib_df.loc[idx-1, 'quarter']

get_recession_start()

'2008q3'

# Terceira Questão: Encontrar o ano de término da recessão

Bom, se precisávamos encontrar o inicio anteriormente, o final é o inicio + 4 trimestres.

In [20]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    trim_df = pd.read_excel('gdplev.xls', skiprows=9, parse_cols='E,G', header=None)
    trim_df.columns = ['quarter', 'GDP']
    trim_df['date'] = trim_df['quarter'].apply(lambda x: x.split('q')[0]).astype(int)
    
    pib_df = trim_df[trim_df['date'] >= 2000]
    pib_df.drop('date', axis=1, inplace=True)
    pib_df.reset_index(inplace=True)
    
    for i in range(0, pib_df.shape[0]-4):
        if ((pib_df.loc[i, 'GDP'] > pib_df.loc[i+1, 'GDP']) and 
            (pib_df.loc[i+1, 'GDP'] > pib_df.loc[i+2, 'GDP']) and 
            (pib_df.loc[i+2, 'GDP'] < pib_df.loc[i+3, 'GDP'])):
            (pib_df.loc[i+3, 'GDP'] < pib_df.loc[i+4, 'GDP'])
            idx = i

    return pib_df.loc[idx+4, 'quarter']

get_recession_end()

'2009q4'

# Quarta questão: Retornar o trimestre de menor PIB durante a recessão

Bom, agora precisamos retornar o ano e trimestre em que houve o _recession bottom_.

Lembrando que uma _recession bottom_ é o trimestre que obteve o menor PIB da recessão.

Podemos repetir o código anterior, e encontrar o min de GDP, dentro dos 4 trimestres;

In [22]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    trim_df = pd.read_excel('gdplev.xls', skiprows=9, parse_cols='E,G', header=None)
    trim_df.columns = ['quarter', 'GDP']
    trim_df['date'] = trim_df['quarter'].apply(lambda x: x.split('q')[0]).astype(int)
    
    pib_df = trim_df[trim_df['date'] >= 2000]
    pib_df.drop('date', axis=1, inplace=True)
    pib_df.reset_index(inplace=True)
    
    for i in range(0, pib_df.shape[0]-4):
        if ((pib_df.loc[i, 'GDP'] > pib_df.loc[i+1, 'GDP']) and 
            (pib_df.loc[i+1, 'GDP'] > pib_df.loc[i+2, 'GDP']) and 
            (pib_df.loc[i+2, 'GDP'] < pib_df.loc[i+3, 'GDP'])):
            (pib_df.loc[i+3, 'GDP'] < pib_df.loc[i+4, 'GDP'])
            idx = i
    
    
    answer = pib_df.iloc[idx-1:idx+4]
    answer.set_index('index', inplace=True)
    answer = answer.loc[answer['GDP'] == answer['GDP'].min(),'quarter']

    return answer.iloc[0]
get_recession_bottom()

'2009q2'

# Quinta questão 

Para essa questão iremos utilizar o ultimo dataset disponibilizado. 

Precisamos converter os dados para trimestres e retornar os valores médios em um dataframe. Esse dataframe precisa ter as colunas com os trimestres (2000q1 ate 2016q3) e com multi-indice com os valores de estado e nome da regiao.

- `City_Zhvi_AllHomes.csv` um .csv contendo todas as casas no nível da cidade, e que possui preços médios de venda de casas em um nível refinado.

Vamos começar carregando-o e entendendo seus dados

In [81]:
df_homes = pd.read_csv('City_Zhvi_AllHomes.csv')
df_homes.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


De antemão podemos notar que a coluna estado possui apenas as siglas deles, mas como nos foi disponibilizado um dicionário para tal, podemos renomar essa coluna. 

Outro fator importante, é que devemos notar que não há uma coluna com os trimestres, mas há cada ano junto com mês. Devemos agrupá-los de 3 em 3.


Ok! Vamos corrigir os estados primeiramente, e já atribuir os índices.

In [82]:
df_homes['State'].replace(states, inplace=True)
df_homes.set_index(['State', 'RegionName'], inplace=True)
df_homes.head()

,,RegionID,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,6181,New York,Queens,1,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
California,Los Angeles,12447,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,154100.0,154300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
Illinois,Chicago,17426,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,109100.0,109000.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
Pennsylvania,Philadelphia,13271,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,49400.0,49300.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
Arizona,Phoenix,40326,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,88500.0,88900.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


---

Certo, já está! Mas vamos recordar que precisamos dos resultados apartir do ano 2000 certo? Vamos selecionar de 2000-01 até 2016-08.

Podemos utilizar um método para selecionar apenas as colunas que vamos utilizar, ```filter```.

Vamos criar uma lista com as colunas que queremos.

---

In [85]:
cols = []
for year in range (2000, 2016):
    for month in range(1, 13):
        cols.append(f"{year}-{month:02d}")
        
# como 2016 vai até mes 8, vamos fazer o ultimo ano separado    
for month in range(1, 9):
        cols.append(f"2016-{month:02d}")
        
df_homes.filter(cols).head()

,,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
California,Los Angeles,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
Illinois,Chicago,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
Pennsylvania,Philadelphia,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
Arizona,Phoenix,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


---

Certo, agora que temos apenas as colunas que utilizaremos, vamos agrupar os trimestres. Podemos criar um dicionário e renomear as colunas com método ````.rename````

---

In [84]:
df_homes = df_homes.filter(cols)

In [91]:
dict_quarters = {}

for year in range (2000, 2016):
    for month in range(1, 13):
        cols.append(f"{year}-{month:02d}")
        if (month <= 3):
            dict_quarters[(f"{year}-{month:02d}")] = (f"{year}q1")
        elif (month >= 4 and month <= 6):
            dict_quarters[(f"{year}-{month:02d}")] = (f"{year}q2")
        elif (month >= 7 and month <= 9):
            dict_quarters[(f"{year}-{month:02d}")] = (f"{year}q3")
        else:
            dict_quarters[(f"{year}-{month:02d}")] = (f"{year}q4")
            
        
# como 2016 vai até mes 8, vamos fazer o ultimo ano separado      
for month in range(1, 9):
    cols.append(f"2016-{month:02d}")
    if (month <= 3):
        dict_quarters[(f"2016-{month:02d}")] = (f"2016q1")
    elif (month >= 4 and month <= 6):
        dict_quarters[(f"2016-{month:02d}")] = (f"2016q2")
    elif (month >= 7 and month <= 9):
        dict_quarters[(f"2016-{month:02d}")] = (f"2016q3")
    else:
        dict_quarters[(f"2016-{month:02d}")] = (f"2016q4")

df_homes.rename(columns=dict_quarters, inplace=True)

---

Certo, já temos o que precisamos, agora só precisamos agrupar por trimestre e contar a média pra cada um.

---

In [87]:
df_homes.groupby(by=df_homes.columns, axis=1).mean().sort_index().head()

2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   

                            2002q2    ...            2015q3         2015q4  \
State   RegionName                    ...                                    
Alabama Adamsville    73133.333333    ...      73933.333333   73866.666667   
        Alabaster    128000.000000    ...     150366.666667  151733.333333   
        Albertville   76366.666667    ...      90233.333333   91366.666667   
        Arab          87700.000000    ...     110900.000000  112233.333333   
        Ardmore                NaN    ...     144600.000000  143966.666667   

                      2016-01   2016-02   2016-03   2016-04   2016-05  \
State   RegionName                                                      
Alabama Adamsville    73600.0   74100.0   74800.0   75200.0   74900.0   
        Alabaster    152800.0  153400.0  154200.0  154900.0  155200.0   
        Albertville   91400.0   92000.0   92600.0   92600.0   92400.0   
        Arab         111000.0  109800.0  109300.0  109500.0  110100.0   
        Ardmore      142200.0  142500.0  143000.0  143200.0  143400.0   

                      2016-06   2016-07   2016-08  
State   RegionName                                 
Alabama Adamsville    74700.0   74700.0   74700.0  
        Alabaster    155200.0  155600.0  156100.0  
        Albertville   92400.0   92300.0   92100.0  
        Arab         110700.0  111600.0  112400.0  
        Ardmore      143100.0  142300.0  141600.0  

[5 rows x 72 columns]

In [92]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    df_homes = pd.read_csv('City_Zhvi_AllHomes.csv')
    df_homes['State'].replace(states, inplace=True)
    df_homes.set_index(['State', 'RegionName'], inplace=True)
    
    cols = []
    for year in range (2000, 2016):
        for month in range(1, 13):
            cols.append(f"{year}-{month:02d}")

    # como 2016 vai até mes 8, vamos fazer o ultimo ano separado      
    for month in range(1, 9):
            cols.append(f"2016-{month:02d}")
            
    df_homes = df_homes.filter(cols)
    dict_quarters = {}

    for year in range (2000, 2016):
        for month in range(1, 13):
            cols.append(f"{year}-{month:02d}")
            if (month <= 3):
                dict_quarters[(f"{year}-{month:02d}")] = (f"{year}q1")
            elif (month >= 4 and month <= 6):
                dict_quarters[(f"{year}-{month:02d}")] = (f"{year}q2")
            elif (month >= 7 and month <= 9):
                dict_quarters[(f"{year}-{month:02d}")] = (f"{year}q3")
            else:
                dict_quarters[(f"{year}-{month:02d}")] = (f"{year}q4")


    # como 2016 vai até mes 8, vamos fazer o ultimo ano separado      
    for month in range(1, 9):
            cols.append(f"2016-{month:02d}")
            if (month <= 3):
                dict_quarters[(f"2016-{month:02d}")] = (f"2016q1")
            elif (month >= 4 and month <= 6):
                dict_quarters[(f"2016-{month:02d}")] = (f"2016q2")
            elif (month >= 7 and month <= 9):
                dict_quarters[(f"2016-{month:02d}")] = (f"2016q3")
            else:
                dict_quarters[(f"2016-{month:02d}")] = (f"2016q4")

    df_homes.rename(columns=dict_quarters, inplace=True)
    df_homes = df_homes.groupby(by=df_homes.columns, axis=1).mean().sort_index()
    
    return df_homes
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State     RegionName                                                     
Alabama   Adamsville        69033.333333   69166.666667   69800.000000   
          Alabaster        122133.333333  123066.666667  123166.666667   
          Albertville       73966.666667   72600.000000   72833.333333   
          Arab              83766.666667   81566.666667   81333.333333   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       81700.000000   78533.333333   79133.333333   
          Bayou La Batre    44066.666667   44500.000000   44266.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54033.333333   54400.000000   54966.666667   
          Boaz              70866.666667   70266.666667   70300.000000   
          Brent             92933.333333   94333.333333   96166.666667   
          Brighton                   NaN            NaN            NaN   
          Brookwood         92566.666667   95100.000000   98866.666667   
          Buhl              90800.000000   94600.000000   96500.000000   
          Calera           108933.333333  110366.666667  108000.000000   
          Center Point      80966.666667   81233.333333   81500.000000   
          Centreville       95300.000000   96566.666667   98000.000000   
          Chalkville        94100.000000   94433.333333   94433.333333   
          Chancellor                 NaN            NaN            NaN   
          Chelsea          162066.666667  167033.333333  166900.000000   
          Chickasaw         51200.000000   53666.666667   54933.333333   
          Chunchula         80266.666667   81766.666667   82200.000000   
          Citronelle        64833.333333   66633.333333   68066.666667   
          Clay             120900.000000  122266.666667  123966.666667   
          Coden             62600.000000   64800.000000   66866.666667   
          Coker            118100.000000  120766.666667  118166.666667   
          Concord           78600.000000   78700.000000   80133.333333   
          Cottondale       100833.333333  102633.333333  104766.666667   
...                                  ...            ...            ...   
Wisconsin Vernon           183200.000000  178200.000000  174300.000000   
          Vienna           178033.333333  181533.333333  182433.333333   
          Vinland          119800.000000  126766.666667  134933.333333   
          Wales                      NaN            NaN            NaN   
          Waterford        121200.000000  119433.333333  120200.000000   
          Waukesha         141266.666667  138433.333333  136733.333333   
          Waunakee         187400.000000  191433.333333  192666.666667   
          Waupun            84000.000000   84000.000000   84400.000000   
          Wausau            69566.666667   69466.666667   71033.333333   
          Wayne                      NaN            NaN            NaN   
          West Allis        80933.333333   81933.333333   84066.666667   
          Weston            80166.666667   82700.000000   84166.666667   
          Wheatland        129800.000000  132133.333333  134933.333333   
          Whitelaw          96033.333333  101433.333333  108033.333333   
          Williams Bay     122900.000000  115300.000000  110766.666667   
          Wilson           129033.333333  129366.666667  132433.333333   
          Wilson                     NaN            NaN            NaN   
          Wind Lake        124366.666667  123666.666667  124133.333333   
          Wind Point       149233.333333  145266.666667  140866.666667   
          Wisconsin Dells  100000.000000  103400.000000  105000.000000   
          Wolf River        95166.666667   99333.333333  103800.000000   
          Woodruff                   N

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"